# Loopless FBA

Here we will use `flux_balance_analysis` and
`flux_variability_analysis` to analyze a toy model of *E. coli* that
is constrained in a way that removes all thermodynamically infeasible loops in
the flux solution. For more details about the algorithm, see [Schellenberger,
and, Palsson., "Elimination of thermodynamically infeasible loops in
steady-state metabolic models.", Biophysical Journal,
2011](https://doi.org/10.1016/j.bpj.2010.12.3707).

If it is not already present, download the model:

In [1]:
!isfile("e_coli_core.xml") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA, GLPK

model = load_model("e_coli_core.xml")

Metabolic model of type SBMLModel
sparse([8, 10, 21, 43, 50, 51, 8, 9, 6, 12  …  33, 66, 68, 72, 23, 26, 33, 72, 22, 33], [1, 1, 1, 1, 1, 1, 2, 2, 3, 3  …  93, 93, 93, 93, 94, 94, 94, 94, 95, 95], [-1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0  …  1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


In COBREXA.jl, the Loopless FBA is implemented as a modification of the
normal FBA, called `add_loopless_constraints`.
We use GLPK optimizer here, because the loopless constraints add integer
programming into the problem. Simpler solvers (such as Tulip) may not be able
to solve the mixed integer-linear (MILP) programs.

In [2]:
loopless_flux = flux_balance_analysis_vec(
    model,
    GLPK.Optimizer,
    modifications = [add_loopless_constraints()],
)

95-element Vector{Float64}:
  0.0
  0.0
  0.0
  6.00724957535027
  6.007249575350215
  0.0
  0.0
  5.06437566148219
  0.0
  0.0
  ⋮
  0.0
  0.0
  5.06437566147912
 -5.06437566148219
  1.4969837572616171
  0.0
  1.4969837572616171
  1.181498093246006
  7.47738196216028

The representation is particularly convenient since it allows to also explore
other properties of loopless models, such as variability and parsimonious
balance, as well as other analyses that accept `modifications` parameter:

In [3]:
loopless_variability = flux_variability_analysis(
    model,
    GLPK.Optimizer,
    modifications = [add_loopless_constraints()],
)

95×2 Matrix{Float64}:
  8.08242e-14   0.0
  8.08242e-14   0.0
  9.91306e-14   0.0
  6.00725       6.00725
  6.00725       6.00725
  9.91306e-14   0.0
  0.0          -4.78665e-13
  5.06438       5.06438
  1.13687e-13   0.0
  7.72715e-14   0.0
  ⋮            
  0.0          -2.23584e-12
  0.0          -2.23584e-12
  5.06438       5.06438
 -5.06438      -5.06438
  1.49698       1.49698
  0.0          -7.69163e-12
  1.49698       1.49698
  1.1815        1.1815
  7.47738       7.47738

For details about the loopless method, refer to Schellenberger, Jan, Nathan
E. Lewis, and Bernhard Ø. Palsson: "Elimination of thermodynamically
infeasible loops in steady-state metabolic models." *Biophysical journal*
100, no. 3 (2011), pp. 544-553.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*